In [2]:
import pickle
import h5py as h5
import numpy as np
import sklearn
import modAL
from modAL.models import ActiveLearner
from sklearn.ensemble import RandomForestClassifier
import collections
import csv

In [46]:
with (open("/scratch/yw3004/sonyc/sonyc_distortion_classification/data/features.pickle", "rb")) as openfile:
    alldata= pickle.load(openfile)

In [47]:
elements = 0
for sensor in alldata.keys():
    elements += len(list(alldata[sensor].values()))

In [48]:
elements

2556316

## Read positive samples file ##

In [44]:
with (open("/scratch/yw3004/sonyc/sonyc_distortion_classification/data/positive_samples.pickle", "rb")) as openfile:
    while True:
        try:
            positive_samples= pickle.load(openfile)
        except EOFError:
            break

In [90]:
positive_samples

{'b827eb429cd4': ['1489667630.26', '1488810598.15'],
 'b827eb5895e9': ['1490690898.85', '1484690200.69'],
 'b827eb0d8af7': ['1484697864.54', '1484651228.29'],
 'b827eb122f0f': ['1498892374.76', '1498261062.11'],
 'b827eb86d458': ['1484713418.23', '1484700558.36'],
 'b827eb0fedda': ['1497928649.89', '1480740512.58'],
 'b827eb8e2420': ['1500149760.47', '1489721873.46'],
 'b827eb1685c7': ['1499724045.58', '1486212620.04'],
 'b827eb815321': ['1482308965.46', '1482399908.63'],
 'b827eb9bed23': ['1491297344.73', '1488939279.51'],
 'b827eb44506f': ['1486149582.98', '1499473078.4'],
 'b827eb4e7821': ['1483014176.71', '1483015339.86'],
 'b827eb2a1bce': ['1484666103.12', '1484674054.82'],
 'b827ebad073b': ['1480303515.84', '1496649071.15'],
 'b827eb42bd4a': ['1490014066.52', '1490032278.74']}

### Get positive samples from features dict ###

In [91]:
totalPositiveEmbedding = np.empty([1,128], dtype=int)
all_sensors = []
timestamp = []
frame = []
for sensor_id in positive_samples.keys():
    for time_stamp in positive_samples[sensor_id]:
        if sensor_id in alldata.keys():
            if time_stamp in alldata[sensor_id].keys():
                embedding = alldata[sensor_id][time_stamp]
                totalPositiveEmbedding = np.vstack((totalPositiveEmbedding, embedding))
                for i in range(embedding.shape[0]):
                    all_sensors.append(sensor_id)
                    timestamp.append(time_stamp)
                    frame.append(i)                    
                    
totalPositiveEmbedding =np.delete(totalPositiveEmbedding, 0, 0) # remove the first dummy 128 embedding

In [92]:
totalPositiveEmbedding.shape

(300, 128)

In [93]:
# add the positive label, 1
positive_xy = np.insert(totalPositiveEmbedding, 128, 1, axis=1)

In [94]:
sensors = np.array(all_sensors)
timestamp = np.array(timestamp)
frames = np.array(frame)

In [95]:
sensors = sensors.reshape((-1, 1))
timestamp = timestamp.reshape((-1,1))
frames = frames.reshape((-1,1))

In [100]:
positive_xy = np.concatenate((positive_xy,sensors),axis=1)
positive_xy = np.concatenate((positive_xy,timestamp),axis=1)
positive_xy = np.concatenate((positive_xy,frames),axis=1)

In [103]:
positive_xy.shape

(300, 132)

In [104]:
positive_xy

array([['145', '10', '124', ..., 'b827eb429cd4', '1489667630.26', '0'],
       ['151', '20', '126', ..., 'b827eb429cd4', '1489667630.26', '1'],
       ['152', '17', '120', ..., 'b827eb429cd4', '1489667630.26', '2'],
       ...,
       ['160', '20', '161', ..., 'b827eb42bd4a', '1490032278.74', '7'],
       ['154', '20', '139', ..., 'b827eb42bd4a', '1490032278.74', '8'],
       ['154', '21', '130', ..., 'b827eb42bd4a', '1490032278.74', '9']],
      dtype='<U21')

In [105]:
with open("/scratch/yw3004/sonyc/sonyc_distortion_classification/data/positive_xy_allInfo.pickle", 'wb') as pfile:
    pickle.dump(positive_xy, pfile, protocol=pickle.HIGHEST_PROTOCOL)

## Read negative samples file ##

In [113]:
negativeCsvData = {}
with open('/scratch/yw3004/sonyc/sonyc_distortion_classification/data/Sensor_data_negative.csv') as csvfile:
    Sensor_data_positive = csv.reader(csvfile, skipinitialspace=True, delimiter=' ')
    csvrow = -1
    for row in Sensor_data_positive:
        if csvrow != -1: # skip the first row
            sid = row[0].split(',')[0]
            ts  = row[0].split(',')[1]
            if sid in negativeCsvData.keys():
                negativeCsvData[sid].append(ts)
            else:
                negativeCsvData[sid] = []
                negativeCsvData[sid].append(ts)
        csvrow += 1   

In [114]:
print(type(negativeCsvData))
print(len(negativeCsvData))
print(negativeCsvData['b827eb4e7821'])

<class 'dict'>
15
['1491211362.82_5', '1491800203.12_8', '1482424939.17_1', '1498868810.69_4', '1488993967.2_1', '1492066899.36_6', '1484297539.06_6', '1490406200.26_1', '1494789341.92_4', '1490926970.82_4', '1488422336.29_2', '1488377458.61_6', '1486479347.85_4', '1483636861.26_6', '1484122883.92_3', '1487404654.04_8', '1492186534.34_2', '1493876183.72_2', '1483134229.53_1', '1491421469.99_7']


In [119]:
totalNegativeEmbedding = np.empty([1,128], dtype=int)
neg_sensors = []
neg_timestamp = []
neg_frame = []
for sensorid in negativeCsvData.keys():
    for timestamp in negativeCsvData[sensorid]:
        if sensorid in alldata.keys():
            timestamp_div = timestamp.split('_')[0]
            embedding_index = timestamp.split('_')[1]
            if timestamp_div in alldata[sensorid]:
                one_negative_embedding = alldata[sensorid][timestamp_div][int(embedding_index)]
                totalNegativeEmbedding = np.vstack((totalNegativeEmbedding, one_negative_embedding))
                neg_sensors.append(sensorid)
                neg_timestamp.append(timestamp_div)
                neg_frame.append(embedding_index)
totalNegativeEmbedding =np.delete(totalNegativeEmbedding, 0, 0) # remove the first dummy 128 embedding

In [120]:
# add the negative label, 0
negative_xy = np.insert(totalNegativeEmbedding, 128, 0, axis=1)

In [123]:
neg_sensors = np.array(neg_sensors)
neg_timestamps = np.array(neg_timestamp)
neg_frames = np.array(neg_frame)

In [124]:
neg_sensors = neg_sensors.reshape((-1, 1))
neg_timestamps = neg_timestamps.reshape((-1,1))
neg_frames = neg_frames.reshape((-1,1))

In [125]:
neg_sensors.shape

(300, 1)

In [126]:
negative_xy = np.concatenate((negative_xy,neg_sensors),axis=1)
negative_xy = np.concatenate((negative_xy,neg_timestamps),axis=1)
negative_xy = np.concatenate((negative_xy,neg_frames),axis=1)

In [131]:
negative_xy

array([['146', '14', '129', ..., 'b827eb4e7821', '1491211362.82', '5'],
       ['154', '30', '171', ..., 'b827eb4e7821', '1491800203.12', '8'],
       ['153', '25', '150', ..., 'b827eb4e7821', '1482424939.17', '1'],
       ...,
       ['158', '29', '166', ..., 'b827eb9bed23', '1495680055.31', '8'],
       ['153', '25', '146', ..., 'b827eb9bed23', '1499458546.46', '6'],
       ['153', '25', '152', ..., 'b827eb9bed23', '1496413283.32', '2']],
      dtype='<U21')

In [135]:
with open("/scratch/yw3004/sonyc/sonyc_distortion_classification/data/negative_xy_allInfo.pickle", 'wb') as pfile:
    pickle.dump(negative_xy, pfile, protocol=pickle.HIGHEST_PROTOCOL)